# 1. 目标检测(Object Detection)

在计算机视觉的实际应用中，分类是远远不够的，更多的场景需要能同时探测多个物体，不仅包括这些物体的分类，而且包括这些物体的位置和尺寸。因此，目标检测是比图像分类更具挑战性的任务。

通常使用包围盒（Bounding Box）来表示目标物体的位置和尺寸。

![img](images/chapter15/object_detection.png)

### 常用的目标检测数据集

- [PASCAL VOC](http://host.robots.ox.ac.uk/pascal/VOC/)
      PASCAL VOC 包含 20 个类别。目前目标检测常用的是 VOC2007 和 VOC2012 数据集。通常使用 VOC2007 和 VOC2012 的训练集和验证集并集（共 16551 张图像）作为训练，用 VOC2007 的测试集（4952 张图像）作为测试。


- [MS COCO](http://cocodataset.org/)
      COCO 中的图片背景比较复杂，目标数量比较多，目标尺寸更小，因此 COCO 数据集上的任务比 VOC 更困难。COCO 包含 80k 训练图像、40k 验证图像和 20k 没有公开标记的测试图像，共 80 个类别，平均每张图包含 7.7 个目标。通常使用 80k 训练和 35k 验证图像的并集作为训练，其余 5k 图像作为验证，20k 测试图像用于线上测试。

### 评价指标

目标检测任务通常使用 IoU（Intersection over Union） 和 mAP 作为模型评价指标。IoU 其定义为模型产生的目标窗口和原来标记窗口的交叠率，即检测结果（预测生成的包围盒）与真实值（Ground Truth）的交集比上它们的并集。因此 IoU 取值范围为 0~1，数值越高代表重叠程度越高。

![img](images/chapter15/IoU.png)

经典的使用卷积神经网络实现目标检测的模型有：

- [R-CNN（2013）](https://arxiv.org/abs/1311.2524)
- [SPPNet（2014）](https://arxiv.org/abs/1406.4729)
- [Fast R-CNN（2015）](https://arxiv.org/abs/1504.08083)
- [Faster R-CNN（2015）](https://arxiv.org/abs/1506.01497)
- [YOLO（2015）](https://arxiv.org/abs/1506.02640)
- [SSD（2015）](https://arxiv.org/abs/1512.02325)

# 2. [R-CNN](https://arxiv.org/abs/1311.2524)

R-CNN（Region-based Convolutional Network） 模型发展到现在已经成为一个系列，从最初的 R-CNN 模型发展到 Fast R-CNN 模型，之后在 2015 年再一次突破，提出了 Faster R-CNN模型。近几年来，基于深度学习的目标检测方法发展很快，相继提出了很多精妙的思想，R-CNN 模型系列功不可没，该模型是很多算法的基础思想。

R-CNN 结合了 [Selective Search](https://www.cs.mcgill.ca/~hsyed2/selectivesearch/Object-Recognition/) 算法的候选区域和由卷积神经网络产生的视觉丰富特征，它的性能比当时最好的检测模型在 Pascal VOC 2012 数据集上有将近 30% 的性能提升，mAP（mean average precision）指标从 40.9% 提升到了 53.3%。

R-CNN 模型的整体框架为：

- 采用 Selective Search 算法生成候选区域（region proposals）
      每张图片大约生成 2000 个候选区域，由于候选区域尺寸不一，所以通过拉伸变换，形成统一的输入尺寸 227×227。

- 运用 CNN 网络进行特征提取
      将统一尺寸的候选区域图像输入 AlexNet 网络并提取 4096 维特征向量
      
- 对候选区域图像进行分类
      使用 SVM 分类器对 CNN 输出特征进行分类
      
- 包围盒回归（Bounding Box Regression）
      用 Bounding Box 回归值校正原来的候选区域窗口，生成预测窗口坐标

![img](images/chapter15/R-CNN.png)

R-CNN 的训练步骤分为四步：

- 在 ImageNet 数据集上进行预训练（pre-training）
![img](images/chapter15/R-CNN_pre_train.png)


- 利用成熟的权重参数在 PASCAL VOC 数据集上进行微调（fine-tune）
      将网络最后一个 1000 个神经元的全连接层改为 21 个神经元
      
![img](images/chapter15/R-CNN_fine_tune.png)
      
- 使用 CNN 提取的特征作为输入信息，为每个分类训练一个 SVM，可以将精度再提高 3.3%

![img](images/chapter15/R-CNN_train_SVM.png)

- 包围盒回归训练

将包围盒回归训练的输入数据记做 $\{(P^i,G^i)\}_{i=1,...,N}$，其中
$$ P^i=(P_x^i, P_y^i, P_w^i, P_h^i) $$
$$ G^i=(G_x^i, G_y^i, G_w^i, G_h^i) $$

$P_x^i$ 和 $P_y^i$ 代表候选区域的中心坐标，$P_w^i$ 和 $P_h^i$ 分别代表候选区域的宽度和高度。$G$ 代表标签包围盒的中心坐标和宽高。为了简化说明，下面省略 $P$ 和 $G$ 的上标 $i$。因此，回归任务的目标即是，给定候选区域 $(Px,Py,Pw,Ph)$ 寻找一种映射 $f$， 使得 $f(P_x,P_y,P_w,P_h)=(\hat G_x,\hat G_y,\hat G_w,\hat G_h)$，并且 $ (\hat G_x,\hat G_y,\hat G_w,\hat G_h) \approx (G_x,G_y,G_w,G_h)$

可以定义映射 $f$ 由平移和缩放组成

$$\begin{cases}
\hat G_x=P_w d_x(P)+P_x\\
\hat G_y=P_h d_y(P)+P_y\\
\hat G_w=P_w \exp(d_w(P))\\
\hat G_h=P_h \exp(d_h(P))
\end{cases}$$

![img](images/chapter15/RCNN-bbox-regression.png)

## 包围盒回归（Bounding Box Regression）

包围盒回归问题就是学习 $d_x(P)$，$d_y(P)$，$d_w(P)$ 和 $d_h(P)$这四个变换。对于该问题，输入是卷积神经网络的输出特征图，定义为 $\phi(P)$；输出是 $d_x(P)$，$d_y(P)$，$d_w(P)$ 和 $d_h(P)$ 四个变换。回归的标签是 $P$ 与 $G$ 之间的变换量，定义为 $t_*$（$*$ 为 $x,y,h,w$ 之一），即

$$\begin{cases}
t_x=(G_x - P_x)/P_w\\
t_y=(G_y - P_y)/P_h\\
t_w=\log(G_w /P_w)\\
t_h=\log(G_h /P_h)
\end{cases}$$

那么目标函数可以表示为：

$$ d_*(P) = W_*^T \phi(P) $$

其中 $W_*^T $ 是需要学习的参数，$d_*(P)$ 是预测值。为了让预测值 $d_*(P)$ 与标签值 $t_*$ 差距最小，设计 L1 损失函数

$$Loss = \sum_i^N |t_*^i - W_*^T \phi(P^i)|$$

函数优化目标为：

$$\hat W_* = argmin_{W_*}\sum_i^N |t_*^i - W_*^T \phi(P^i)|+\lambda ||W_*||$$

在测试阶段，R-CNN 使用训练好的 CNN 和 SVM 对每张图像的约 2000 个候选区域进行分类并排序，再使用 **非极大值抑制算法**（non-maximum suppression）去除重叠区域，得到最终预测结果。

## 非极大值抑制(Non-Max Suppression, NMS)

目标检测可能会出现的一个问题是，模型会对同一目标做出多次预测，得到多个包围盒。NMS 旨在保留最接近真实包围盒的那一个预测结果，而抑制其它相似的预测结果。

NMS 算法流程如下：

- 对每个类别统计每个包围盒预测结果输出的属于该类别概率，并将预测结果按该概率由高至低排序。


- 对应概率很小的预测结果认为其没有找到目标，通过设定合理阈值将其移除


- 在剩余的预测结果中，找到对应概率最大的预测结果，将其输出，并移除和该包围盒有很大重叠（如IoU大于 0.3）的其它包围盒


- 重复上一步，直到所有的预测结果均被处理

R-CNN 模型由于提出较早，相比于目前的目标检测网络模型，精确度不够高，速度也较慢，无法满足实时系统需要。但 R-CNN 让业界第一次看到了深度学习在目标检测上的力量和潜力，是深度学习在目标检测任务上迈出的重要一步。

# 3. [Fast R-CNN](https://arxiv.org/abs/1504.08083)

R-CNN 模型主要存在以下三个问题：

- 测试速度慢

      R-CNN 将一张图像分解成近 2000 个候选区域，每个候选区域拉伸形成的图像都会单独通过 CNN 提取特征。实际上这些候选区域之间大量重叠，特征值之间完全可以共享，因此造成了运算能力的浪费
      
- 训练速度慢
      R-CNN 在训练时，在应用 SVM 分类之前，将 CNN 提取的特征存储在硬盘上，在硬盘上大量读写交换数据造成训练速度缓慢
      
- 训练所需空间大
      R-CNN 中独立的 SVM 分类器和回归器需要大量特征作为训练样本，需要存储上千GB的数据

R-CNN 的原作者 Ross Girshick 在吸收了 [SPPNet](https://arxiv.org/abs/1406.4729) 的优点后再一次推出了 R-CNN 的继承者 Fast R-CNN。Fast R-CNN 构思精巧，流程更为紧凑，大幅提升了目标检测的速度。与 R-CNN 相比，训练时间从 84 小时减少为 9.5 小时，测试时间从 47 秒减少为 0.32 秒，在 PASCAL VOC 2007 数据集上的准确率为 66%。

Fast R-CNN 模型整体训练框架大致为：

- 将整张图像输入至 CNN 网络，进行特征提取


- 使用 Selective Search 算法生成候选区域，并将候选区域窗口映射到 CNN 的最后一层卷积特征图上


- 通过 ROI（Regions of Interest） Pooling 层将每个候选区域窗口生成固定尺寸的特征图


- 将特征图输入至全连接层网络提取 ROI 特征向量


- Softmax 层对 ROI 特征向量进行分类，回归层使用 Smooth L1 损失函数进行包围盒回归，并加权联合二者损失函数值

$$Smooth_{L1}(x)=
\begin{cases}
0.5x^2&x < 1 \\
|x|-0.5&otherwise
\end{cases}$$

![img](images/chapter15/Fast_R-CNN.png)

由于用于目标检测的数据集较小，无法充分从零开始训练网络，Fast R-CNN 和 R-CNN 一样采用在预训练网络模型上微调参数的方法，即通过 ImageNet 数据集对 CNN 进行预训练，然后用 PASCAL VOC 数据集再进行微调训练。Fast R-CNN 使用了三种预训练网络模型，分别是小型网络 CaffeNet（AlexNet 修改版）、中型网络 VGG_VNN_M_1024（和 AlexNet 同样深度的 VGG 网络）和大型网络 VGG16。

另外，Fast R-CNN 验证了使用 Softmax 层可以实现比 SVM 更好的分类性能：

|模型|分类器|CaffeNet|VGG_VNN_M_1024|VGG16|
|---|----|---|--|--|
|Fast R-CNN|SVM|56.3|58.7|66.8|
|Fast R-CNN|Softmax|**57.1**|**59.2**|**66.9**|

# 4. [Faster R-CNN](https://arxiv.org/abs/1506.01497)

虽然 Fast R-CNN 基于 R-CNN 有很大的改进，但 Fast R-CNN 仍然基于 Selective Search 算法提取候选区域，而 Selective Search 算法提取候选区域是基于 CPU 运行的，无法借用 GPU 的高度并行运算能力，因此效率依然不高。另外，这种方法为每幅图像提供大约 2000 个候选区域，如此多的候选区域加重了后面神经网络的计算压力。

Faster R-CNN 在吸取了Fast R-CNN 特点的前提下，将目标检测的四个基本步骤（候选区域生成、特征提取、分类和包围盒回归）统一到一个深度神经网络框架之内。Faster R-CNN 加入了提取候选区域的神经网络 **RPN**（Region Proposal Network），用 RPN 直接预测出候选区域框，数据限定在 300 个。RPN 的预测在 GPU 中完成，并且卷积网络和 Faster R-CNN 部分共享，因此大幅提升了目标检测的速度。

![img](images/chapter15/Faster_R-cnn_structure.png)

Faster R-CNN 模型的整体框架如下：

- 将整张图像输入 CNN 进行特征提取


- 使用 RPN 为每张图像生成 300 个候选区域


- 将候选区域映射到 CNN 的最后一层卷积特征图上


- 通过 ROI Pooling 层将每个候选区域特征图变成固定尺寸的特征图


- 使用 Softmax 交叉熵损失和 Smooth L1 损失对分类概率和包围盒回归联合训练

Faster R-CNN 网络结构如下图所示

![img](images/chapter15/Fast_R-CNN.jpg)

在生成候选区域的 RPN 子网络设计中，Faster R-CNN 在 CNN 输出的特征图上运用 3×3 滑动窗口（使用 3×3 卷积实现），输出 256 维特征向量。这些向量被输入到两个分支，分别用来分类预测和包围盒回归。

为了使得每个窗口可以检测多个目标，在每一个滑动窗口的位置，同时预测 $k$ 个**锚框**（anchor box），因此回归层（reg layer）生成 $4k$ 个数值代表 $k$ 个锚框的坐标；分类层（cls layer）生成 $2k$ 个预测结果，代表 $k$ 个锚框包含或不包含物体的概率。

![img](images/chapter15/region_proposal_network.png)

为了能够应对不同尺寸的物体，Faster R-CNN 采用了 3 种不同长宽比的锚框（长宽比分别是 1:1，2:1 和 1:2），并有 3 个不同尺度，如下图所示。因此共采用了 9 种锚框（$k=9$）。具体作法是：将每个特征点映射到原始图像的感受野的中心点当成一个基准点，然后围绕这个基准点选取 $k$ 个不同缩放比和长宽比的候选区域。

![img](images/chapter15/anchors.jpg)

Faster R-CNN 与 Fast R-CNN 性能对比：

|模型|网络|卷积运算|候选区域生成|逐区域|合计|帧速度|
|---|---|----|----|----|---|---|
|Fast R-CNN|VGG16|146 ms|1510 ms|174 ms|1830 ms| 0.5 fps|
|Faster R-CNN|VGG16|141 ms|10 ms|47 ms|198 ms|5 fps|

Faster R-CNN 是深度学习算法在目标检测任务上的重大进步，让业界看到了深度学习算法在目标检测任务上进行 **实时处理** 的希望。

# 5. [YOLO](https://arxiv.org/abs/1506.02640)

2015年，随着 **YOLO**（You Only Look Once） 算法的出现，深度学习目标检测算法开始有了两步（two-stage）和单步（single-stage）之分。区别于以 R-CNN 系列为代表的两步检测算法，YOLO 舍去了候选区域提取分支，直接将特征提取、候选框回归和分类在同一个无分支的卷积网络中完成，使得网络结构变得简单，检测速度较 Faster R-CNN 也有近 10 倍的提升，这使得深度学习目标检测算法开始能够满足实时检测任务的需求。

YOLO 模型的处理过程很简单，

- 将输入图像缩放至 448×448

- 将图像输入至单一的 CNN

- 运行非极大值抑制

![img](images/chapter15/YOLO_detection_system.png)

- 将输入图像划分成 $S*S$ 的网格，如果目标物体的中心落在网格内，则由该网格负责检测这个物体


- 每个网格都预测 B 个包围盒，每个包围盒包含 5 个参数，分别是包围盒中心坐标和宽高 $(x, y, w, h)$ 以及置信度。置信度代表包围盒有物体的概率以及和标签的 IoU，可以记做 $\text{Pr}(Object)*\text{IoU}_{pred}^{truth}$


- 每个网格都预测在所有类别上的概率，记做 $C =\text{Pr}(Class|Object)$


- 网络最终的输出为 $S\times S\times(B*5+C)$个张量。YOLO 模型用来预测 PASCAL VOC 数据集时，$S=7$，$B=2$，$C=20$，因此最终输出为 $7\times7\times 30$ 的张量


- 使用非极大值抑制去除冗余包围盒

![img](images/chapter15/YOLO_model.png)

YOLO 模型的网络架构受到了用于图像分类的 GoogLeNet 模型的启发。该网络包含 24 个卷积层和 2 个全连接层，但是没有使用 Inception 模块，而是用 1×1 和 3×3 的卷积层来代替。网络的最后两层为全连接层，最终输出为 $7\times7\times 30$ 的特征图。

![img](images/chapter15/YOLO_architecture.png)

网络的损失函数定义如下，

$$\begin{align*}
Loss=&\lambda_{coord} \sum_{i=0}^{S^2}\sum_{j=0}^{B} \mathbb{1}_{ij}^{obj}[(x_i-\hat x_i)^2+(y_i-\hat y_i)^2]\\
+&\lambda_{coord} \sum_{i=0}^{S^2}\sum_{j=0}^{B} \mathbb{1}_{ij}^{obj}[(\sqrt{w_i}-\sqrt{\hat w_i})^2+(\sqrt{h_i}-\sqrt{\hat h_i})^2]\\
+&\sum_{i=0}^{S^2}\sum_{j=0}^{B} \mathbb{1}_{ij}^{obj}(C_i-\hat C_i)^2\\
+&\lambda_{noobj} \sum_{i=0}^{S^2}\sum_{j=0}^{B} \mathbb{1}_{ij}^{noobj}(C_i-\hat C_i)^2\\
+&\sum_{i=0}^{S^2}\mathbb{1}_{i}^{obj}\sum_{c\in classes}(p_i(c)-\hat p_i(c))^2
\end{align*}$$

损失函数前两项为包围盒坐标和尺寸预测，第3、4项为包含物体和不包含物体的置信度预测，最后一项为类别预测。

其中，$\mathbb{1}_{i}^{obj}$ 代表物体出现在网格 $i$ 中，$\mathbb{1}_{ij}^{obj}$ 代表网格 $i$ 的第 $j$ 个包围盒。$\lambda_{coord}$ 和 $\lambda_{noobj}$ 分别代表包含物体和不包含物体的包围盒权重。



由于多数候选包围盒其实并不包含物体，所以通常设置 $\lambda_{coord}=5$，$\lambda_{noobj}=0.5$。另外，如果对不同尺寸的包围盒赋予同样的权重同样不利于检测小物体，所以在损失函数中使用包围盒宽高的平方根代替宽高。

YOLO 将目标检测任务转换成一个回归问题，极大加快了检测的速度，使得 YOLO 模型可以每秒处理45张图像。

|模型|mAP|帧速率FPS|
|---|----|-----|
|Fast R-CNN|70.0|0.5|
|Faster R-CNN|73.2|7|
|YOLO VGG16|66.4|21||
|YOLO |63.4|45|
|Fast YOLO|52.7|155|

YOLO 模型的缺点：

- 仅将图像划分为 7x7 的网格，每个网格只产生两个包围盒，并且只预测一类物体。这限制了相邻物体的识别能力，无法区分当多个目标出现在同一个网格的情形


- 模型从训练数据中学习了包围盒形状，因此识别同一类物体的新的或不常见的长宽比泛化能力较低


- 模型使用经过多次降采样、相对低分辨率的特征图来预测包围盒，因此包围盒边框识别精度有待进一步提高

# 5. [SSD](https://arxiv.org/abs/1512.02325)

**SSD**（Single Shot Detector） 对 YOLO 进行了改进，达到了和两步方法相当的精度，同时又保持了较快的运行速度。SSD 也采用了网格划分的思想，与 Faster R-CNN 不同的是，它将所有的操作整合在一个卷积网络中完成。相比 YOLO，SSD 在卷积特征后加了若干卷积层以减小特征空间大小，并通过综合多层卷积层的检测结果以检测不同大小的目标。

SSD 采用 VGG16 作为基础模型，将 VGG16 的全连接层 FC6 和 FC7 转换成 $3\times3$ 卷积层 Conv6 和 $1\times1$ 卷积层 Conv7，并在 VGG16 的基础上新增了卷积层进一步实现降采样，以此来获得更多尺度的特征图用于检测。较大的特征图来用来检测相对较小的目标，而小的特征图负责检测大目标。

![img](images/chapter15/SSD_vs_YOLO.png)

SSD 在特征图的每个单元设置尺度和长宽比不同的 4 个先验框（default box）

![img](images/chapter15/SSD_framework.png)

SSD 模型与经典模型精度和速度对比：

|模型|输入图像分辨率|mAP|帧速率FPS|
|---|-------|----|---|
|Faster R-CNN(VGG16)|1000×600|73.2|7|
|YOLO(VGG16)|448×448|66.4|21|
|Fast YOLO|448×448|52.7|155|
|SSD300|300×300|74.3|59|
|SSD512|512×512|**76.8**|22|